In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

## 依照指示取出模型特定層的資訊

In [2]:
# Define model
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16* 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model
model = TheModelClass()


In [3]:
model.children

<bound method Module.children of TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)>

In [4]:
for name, _ in model.named_children():
    print(name)

conv1
pool
conv2
fc1
fc2
fc3


### 取出 self.pool層兩次的輸出，包含：
* x = self.pool(F.relu(self.conv1(x)))
* x = self.pool(F.relu(self.conv2(x)))

In [5]:
outputs= []
def layer1_hook(module, input_, output):
    outputs.append(output)
    
model.pool.register_forward_hook(layer1_hook)

In [6]:
input_ = torch.randn(1, 3, 32, 32)
output = model(input_)

In [7]:
outputs

[tensor([[[[6.7872e-01, 3.8701e-01, 6.4865e-01,  ..., 4.7423e-01,
            5.9514e-01, 1.4456e-01],
           [1.0467e+00, 7.4821e-01, 2.7500e-01,  ..., 1.2104e-01,
            1.1606e+00, 6.0902e-01],
           [1.4532e+00, 5.3453e-01, 1.1349e+00,  ..., 8.3511e-02,
            7.9103e-01, 4.6950e-01],
           ...,
           [8.0318e-01, 4.1324e-02, 9.0125e-01,  ..., 2.6319e-01,
            7.5600e-01, 9.3452e-01],
           [9.7012e-01, 3.7106e-01, 2.5472e-01,  ..., 3.5989e-01,
            6.4281e-01, 0.0000e+00],
           [5.6278e-01, 2.8423e-01, 8.9889e-01,  ..., 3.5510e-01,
            1.2368e+00, 0.0000e+00]],
 
          [[8.3596e-01, 1.0272e+00, 6.0102e-02,  ..., 5.2000e-01,
            1.1595e+00, 3.0944e-01],
           [9.3091e-01, 8.4790e-01, 1.2889e+00,  ..., 7.5508e-01,
            5.3127e-01, 1.0889e+00],
           [1.1445e+00, 6.8914e-01, 1.1049e+00,  ..., 0.0000e+00,
            1.0112e+00, 7.7755e-01],
           ...,
           [3.9940e-01, 6.4249e-01, 6.

In [8]:
print(outputs[0].shape)
print(outputs[1].shape)

torch.Size([1, 6, 14, 14])
torch.Size([1, 16, 5, 5])


## 加入自定義 initialization fuction

#### 對所有Conv2D層使用自定義initialization function
* weight : nn.init.kaiming_normal_
* bias : 全部輸入1

In [9]:
from torch.nn import init

def weights_init(m):
    #classname = m.__class__.__name__
    if isinstance(m, nn.Conv2d):
        #torch.nn.init.xavier_uniform_(m.weight)
        nn.init.kaiming_normal_(m.weight.data,
                                    a=0,
                                    mode='fan_out',
                                    nonlinearity='relu')
        if m.bias is not None:
            torch.nn.init.ones_(m.bias)
            
model.apply(weights_init)

TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

#### 查看 conv層的bias是否皆為1

In [10]:
for name, parameters in model.named_parameters():
    if ('conv' in name) and ('bias' in name):
        print(name, parameters)
        print('\n')

conv1.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1.], requires_grad=True)


conv2.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       requires_grad=True)


